In [144]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from portfolio_functions import *

In [145]:
#load dataframe
df = pd.read_csv("gp_data_1986_to_2015.csv")
#preview
df.head()

,Month,Mkt-RF,RF,Agric,Food,Soda,Beer,Smoke,Toys,Fun,...,BusSv,Comps,Chips,LabEq,Paper,Boxes,Trans,Whlsl,Rtail,Meals
0,198601,0.65,0.56,7.92,2.38,-1.20,-0.86,5.55,2.14,3.91,...,0.78,-1.11,0.85,3.80,4.61,1.91,5.60,2.48,1.67,-0.09
1,198602,7.13,0.53,13.98,7.89,11.68,7.71,12.20,10.72,8.28,...,7.47,2.71,8.34,9.98,8.68,10.61,7.03,8.85,7.21,9.27
2,198603,4.88,0.60,2.74,7.84,10.89,9.17,11.52,5.65,6.86,...,4.96,-0.11,0.04,-1.15,4.18,5.61,3.22,7.41,9.11,6.42
3,198604,-1.31,0.52,5.24,-0.57,4.44,-2.92,3.64,3.91,8.36,...,3.33,3.87,3.55,2.69,-2.50,-0.93,-2.64,1.70,-0.70,3.86
4,198605,4.62,0.49,1.08,8.55,7.95,6.37,10.16,3.16,5.91,...,5.41,1.01,-0.75,2.51,6.66,5.07,0.70,5.52,11.95,4.89


## 1. Data Exploration and Analysis

First, we will reinforce the tools we have learned in the class and construct diversified portfolios among
the 43 industries.

### 1.1. Pre-processing

Calculate the excess return of the 43 industry portfolios. (Note that the excess return of the
market portfolio is already provided.) Subsequently, we will work with the excess returns directly
and treat the riskless rate as zero. Also, you may also treat each industry portfolio as an
individual asset.


In [146]:
#directly convert original monthly returns to excess returns; subtract rf from rf so risk-free rate is 0
df.iloc[:, 2:] = df.iloc[:, 2:].sub(df.iloc[:, 2], axis = 0)
df.head()

,Month,Mkt-RF,RF,Agric,Food,Soda,Beer,Smoke,Toys,Fun,...,BusSv,Comps,Chips,LabEq,Paper,Boxes,Trans,Whlsl,Rtail,Meals
0,198601,0.65,0.0,7.36,1.82,-1.76,-1.42,4.99,1.58,3.35,...,0.22,-1.67,0.29,3.24,4.05,1.35,5.04,1.92,1.11,-0.65
1,198602,7.13,0.0,13.45,7.36,11.15,7.18,11.67,10.19,7.75,...,6.94,2.18,7.81,9.45,8.15,10.08,6.50,8.32,6.68,8.74
2,198603,4.88,0.0,2.14,7.24,10.29,8.57,10.92,5.05,6.26,...,4.36,-0.71,-0.56,-1.75,3.58,5.01,2.62,6.81,8.51,5.82
3,198604,-1.31,0.0,4.72,-1.09,3.92,-3.44,3.12,3.39,7.84,...,2.81,3.35,3.03,2.17,-3.02,-1.45,-3.16,1.18,-1.22,3.34
4,198605,4.62,0.0,0.59,8.06,7.46,5.88,9.67,2.67,5.42,...,4.92,0.52,-1.24,2.02,6.17,4.58,0.21,5.03,11.46,4.40


### 1.2. Basic Portfolio Construction and In-sample analysis

1. Suppose you are asked to allocate your investment among the 43 industries. Use the 1986-2015
historical data, construct the following three portfolios:
<br>
• The equally weighted portfolio (EWP);
<br>
• The tangency portfolio (TAN);
<br>
• The global minimum variance portfolio (GMV).

In [147]:
#calculate expected return and covariance matrix
expected_rtn = estimate_mu(df.iloc[:, 3:])
cov_matrix = estimate_V(df.iloc[:, 3:])

#form portfolios
tangency_wt = tangency(expected_rtn, cov_matrix)
gmv_wt = gmv(cov_matrix)
ewp_wt = ewp(43)

2. Calculate the realized (in-sample) expected returns, standard deviations, Sharpe ratios, and
betas of every 43 industry portfolio, as well as the market portfolio (MKT), EWP, TAN, and
GMV.

In [174]:
#calculate monthly returns for each portfolio
df["Tangency"] = df.iloc[:, 3:46] @ tangency_wt
df["GMV"] = df.iloc[:, 3:46] @ gmv_wt
df["EWP"] = df.iloc[:, 3:46] @ ewp_wt

#dictionary of portfolio weights to be used later
wt = {
    "43": tangency_wt,
    "44": gmv_wt,
    "45": ewp_wt
}

output = {}

#calculate sigma for each asset (excl mkt)
for i in range(46):
    col = df.columns[i + 3]
    cov = EmpiricalCovariance().fit(df.iloc[:, [1, i + 3]]).covariance_
    beta = cov[0, 1]/cov[0,0]

    if i < 43:
        portfolio_wt = np.zeros(43)
        portfolio_wt[i] += 1
        perf_summary  = evaluate_portfolio_performance_on_data(portfolio_wt, df.iloc[:, 3:46])
    else:
        perf_summary = evaluate_portfolio_performance_on_data(wt[f"{i}"], df.iloc[:, 3:46])

    output[col] = [perf_summary['Er'], perf_summary['sigma'], perf_summary['Sharpe'], beta]

#calculate stats for market portfolio
Er_mkt = df["Mkt-RF"].mean()
Sigma_mkt = df.iloc[:, 1].std(ddof = 0)
Sharpe_mkt = Er_mkt/Sigma_mkt
beta_mkt = 1
output["Mkt"] = [Er_mkt, Sigma_mkt, Sharpe_mkt, beta_mkt]

#output dataframe
pd.DataFrame.from_dict(output, orient = 'index', columns = ['Er', 'Sigma', 'Sharpe', 'Beta'])

,Er,Sigma,Sharpe,Beta
Agric,0.700861,6.391212,0.109660,0.781294
Food,0.836806,4.446420,0.188198,0.599901
Soda,0.899500,6.992383,0.128640,0.825378
Beer,0.991889,5.099963,0.194489,0.639299
Smoke,1.254278,6.838366,0.183418,0.632916
Toys,0.531028,6.697157,0.079292,1.044329
Fun,0.940722,7.585837,0.124010,1.344289
Books,0.480139,5.717813,0.083972,1.030524
Hshld,0.667972,4.492962,0.148671,0.716603
Clths,0.822778,6.565338,0.125321,1.082334
